In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [19]:
DATA_FOLDER = 'data/'
WIKISPEEDIA_PATHS = DATA_FOLDER + "wikispeedia_paths-and-graph/"

ARTICLES = WIKISPEEDIA_PATHS + "articles.tsv"
CATEGORIES = WIKISPEEDIA_PATHS + "categories.tsv"
LINKS = WIKISPEEDIA_PATHS + "links.tsv"
PATHS_FINISHED = WIKISPEEDIA_PATHS + "paths_finished.tsv"
PATHS_UNFINISHED = WIKISPEEDIA_PATHS + "paths_unfinished.tsv"
SHORTEST_PATHS = WIKISPEEDIA_PATHS + "shortest-path-distance-matrix.txt"
FEMALES = DATA_FOLDER + "listwomen.txt"
ALL_PATHS = DATA_FOLDER + "all_paths_overview.csv"
PEOPLE_OVERVIEW = DATA_FOLDER + "people_overview.csv"

In [20]:
articles = pd.read_csv(ARTICLES, sep='\t', names=["article"], skiprows = 12)
categories = pd.read_csv(CATEGORIES, sep='\t', names=["article", "category"], skiprows = 13)
links = pd.read_csv(LINKS, sep='\t', names=["link_source", "link_target"], skiprows = 12)
paths_finished = pd.read_csv(PATHS_FINISHED, sep='\t', names=["hashedIpAddress", "timestamp", 
                                                              "durationInSec", "path", "rating"], skiprows = 15)
paths_unfinished = pd.read_csv(PATHS_UNFINISHED, sep='\t', names=["hashedIpAddress", "timestamp", 
                                                                  "durationInSec", "path", "target", "type"], skiprows = 17)
shortest_distance_text = np.loadtxt(SHORTEST_PATHS, dtype=str, comments="#", unpack=False)
female = pd.read_csv(FEMALES, sep=',', names=["name", "women_label"], skiprows = 1)
all_paths_overview = pd.read_csv(ALL_PATHS).drop(columns = ['Unnamed: 0'])
people_overview = pd.read_csv(PEOPLE_OVERVIEW)

Choose subsets of all_paths_overview as the data points. 

,source,target,finished,human_path_length,shortest_path_length,durationInSec,type,failure_article,rating,backclicks,hashedIpAddress,timestamp
0,14th_century,African_slave_trade,True,9,3,166,NaN,NaN,NaN,0,6a3701d319fc3754,1297740409
1,14th_century,African_slave_trade,True,5,3,88,NaN,NaN,3.0,0,3824310e536af032,1344753412
2,14th_century,African_slave_trade,True,8,3,138,NaN,NaN,NaN,0,415612e93584d30e,1349298640
3,14th_century,Greece,True,4,2,37,NaN,NaN,NaN,0,64dd5cd342e3780c,1265613925
4,14th_century,John_F._Kennedy,True,7,3,175,NaN,NaN,3.0,0,015245d773376aab,1366730828
...,...,...,...,...,...,...,...,...,...,...,...,...
76150,Franz_Kafka,Cholera,False,8,3,180,restart,Ghana,NaN,1,109ed71f571d86e9,1389787605
76151,Modern_history,Hollandic,False,1,3,6,restart,Modern_history,NaN,0,232f992e57d43e8d,1389787697
76152,Computer_programming,The_Beatles,False,5,3,1900,timeout,<,NaN,1,2e09a7224600a7cd,1389798400
76153,Jamaica,Alan_Turing,False,4,3,1903,timeout,Battle_of_Midway,NaN,0,60af9e2138051b96,1389799481
